In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
def percMissing(df):
    #Verify Missing columns
    s = pd.Series()
    for col in df.columns:
        s.at[col] = ((len(df) - df[col].count()) / len(df)) * 100
        df_missing = pd.DataFrame({'col':s.index, 'perc_missing':s.values})

    return df_missing.sort_values(by=['perc_missing'], ascending=False)

In [3]:
def count_unique(df, cols):
    for col in cols:
        print(df[col].value_counts(), '\n')

In [17]:
deliveries = pd.read_csv('Greenly Case Study - customerio_deliveries.csv', sep=',')#missing all col event_id, newsletter_id, content_id 
attributes = pd.read_csv('Greenly Case Study - customerio_attributes.csv', sep=',')#no missing col
metrics = pd.read_csv('Greenly Case Study - customerio_metrics.csv', sep=',')#missing col reason(95.5%), link_id(93), link_url(93)
people = pd.read_csv('Greenly Case Study - customerio_people.csv', sep=',')#missing col customer_id(29)


connection = sqlite3.connect('database.db')
deliveries.to_sql('deliveries', connection, if_exists='replace')
attributes.to_sql('attributes', connection, if_exists='replace')
metrics.to_sql('metrics', connection, if_exists='replace')
people.to_sql('people', connection, if_exists='replace')

1000

In [141]:
deliveries.head(10)

,worskpace_id,delivery_id,internal_customer_id,subject_id,event_id,delivery_type,campaign_id,action_id,newsletter_id,content_id,created_at,seq_num
0,123456,dgT33AcAAI-2JY62JQGG2-f84M6sRo5E_D6ChHg=,f7dc0714ce668fb625,01GV679R018Y0ZBSYMJFA9ZQRG,NaN,email,106,1073,NaN,NaN,2023-03-13 16:57:39.000000 UTC,33413
1,123456,dgT33AcAAI-2JY62JQGGzHTgAZyGyMm1lZtmQDc=,f7dc0714ce668fb625,01GV679R018Y0ZBSYMJFA9ZQRG,NaN,email,106,986,NaN,NaN,2023-03-10 16:57:37.000000 UTC,28652
2,123456,dgT33AcAAI-3JY63JQGF_ekTiA40R-5DgdrdHBI=,f7dc0714ce678fb725,01GQSSRN6BMVDSY0AKP5PWF3XH,NaN,email,67,454,NaN,NaN,2023-01-29 14:23:08.000000 UTC,8104
3,123456,dgT33AcAAI-5Io65IgGFpXkArFcmB3VOYR4zgJ0=,f7dc0712aebe018fb922,01GPJQJ05C1S5HZWC6HVT857BC,NaN,email,77,547,NaN,NaN,2023-01-12 10:14:08.000000 UTC,4676
4,123456,dgT33AcAAI-5Io65IgGFtqdnxb9Ig25HuYgnv94=,f7dc0712aebe018fb922,01GPP5N8NZFR9X4XR10M2MK7ZG,NaN,email,67,453,NaN,NaN,2023-01-15 18:18:22.000000 UTC,6259
5,123456,dgT33AcAAI-5JY65JQGG2-f84Ma0wbQOVeaOYzU=,f7dc0714ce698fb925,01GV679R02S7MGT4WQR3B9SMNY,NaN,email,106,1073,NaN,NaN,2023-03-13 16:57:39.000000 UTC,33419
6,123456,dgT33AcAAI-5JY65JQGGzHTgAsU-aeT4_Id_qZw=,f7dc0714ce698fb925,01GV679R02S7MGT4WQR3B9SMNY,NaN,email,106,986,NaN,NaN,2023-03-10 16:57:37.000000 UTC,28670
7,123456,dgT33AcAAI-6JY66JQGGM0-72sAeK8BimbaxSEo=,f7dc0714ce6a8fba25,01GRMG5XPBWHZ0ADV1PCAY8YJ7,NaN,email,67,453,NaN,NaN,2023-02-08 23:15:08.000000 UTC,11451
8,123456,dgT33AcAAI-ACY6ACQGFpa_2bEPT5hVEvHJ3PiY=,f7dc0704ced6018f8009,01GPJQHZZQSD6G0DH49V9AW701,NaN,email,77,548,NaN,NaN,2023-01-12 11:14:10.000000 UTC,5855
9,123456,dgT33AcAAI-AJo6AJgGG-W1m-57MTJyoCL-ICYM=,f7dc0714ceb0018f8026,01GVMZMMJPHA2M5XS21C5CXSHW,NaN,email,16,314,NaN,NaN,2023-03-19 10:32:22.000000 UTC,45341


In [104]:
query = '''
SELECT campaign_id, COUNT(*) FROM deliveries
GROUP BY campaign_id
'''
f = pd.read_sql_query(query, connection)
f

,campaign_id,COUNT(*)
0,16,621
1,17,5
2,47,29
3,64,14
4,67,28
5,68,7
6,69,13
7,71,8
8,77,90
9,84,3


In [198]:
query1 = '''
SELECT 
  d.campaign_id,
  SUBSTRING(d.created_at, 1, 10) AS date_sent,
  COUNT(DISTINCT CASE WHEN m.metric = 'sent' THEN d.delivery_id END) AS num_emails_sent,
  COUNT(DISTINCT CASE WHEN m.metric = 'delivered' THEN d.delivery_id END) / 
    NULLIF(COUNT(DISTINCT CASE WHEN m.metric = 'sent' THEN d.delivery_id END), 0) AS deliverability_rate,
  COUNT(DISTINCT CASE WHEN m.metric IN ('bounced') THEN d.delivery_id END) / 
    NULLIF(COUNT(DISTINCT CASE WHEN m.metric = 'sent' THEN d.delivery_id END), 0) AS bounce_rate,
  COUNT(DISTINCT CASE WHEN m.metric = 'opened' THEN d.delivery_id END) / 
    NULLIF(COUNT(DISTINCT CASE WHEN m.metric = 'delivered' THEN d.delivery_id END), 0) AS open_rate,
  COUNT(DISTINCT CASE WHEN m.metric = 'clicked' THEN d.delivery_id END) / 
    NULLIF(COUNT(DISTINCT CASE WHEN m.metric = 'delivered' THEN d.delivery_id END), 0) AS click_rate,
  COUNT(DISTINCT CASE WHEN m.metric = 'clicked' THEN d.delivery_id END) / 
    NULLIF(COUNT(DISTINCT CASE WHEN m.metric = 'opened' THEN d.delivery_id END), 0) AS reactivity_rate,
  COUNT(DISTINCT CASE WHEN m.metric = 'unsubscribed' THEN d.delivery_id END) / 
    NULLIF(COUNT(DISTINCT CASE WHEN m.metric = 'sent' THEN d.delivery_id END), 0) AS unsubscribe_rate
FROM 
  deliveries d
JOIN 
  metrics m ON d.delivery_id = m.delivery_id
GROUP BY 
  d.campaign_id,
  date_sent;
'''
query = '''
SELECT 
  d.campaign_id,
  SUBSTRING(d.created_at, 1, 10) AS date_sent,
  COUNT(DISTINCT CASE WHEN m.metric = 'sent' THEN d.delivery_id END) AS num_emails_sent,
  COUNT(DISTINCT CASE WHEN m.metric = 'delivered' THEN d.delivery_id END) AS num_deliveried,
  COUNT(DISTINCT CASE WHEN m.metric IN ('bounced') THEN d.delivery_id END) AS num_bounced,
  COUNT(DISTINCT CASE WHEN m.metric = 'opened' THEN d.delivery_id END) AS num_opened,
  COUNT(DISTINCT CASE WHEN m.metric = 'clicked' THEN d.delivery_id END) AS clicked,
  COUNT(DISTINCT CASE WHEN m.metric = 'unsubscribed' THEN d.delivery_id END) AS num_unsubscribed
FROM 
  deliveries d
JOIN 
  metrics m ON d.delivery_id = m.delivery_id
GROUP BY 
  d.campaign_id,
  date_sent;
'''

f = pd.read_sql_query(query, connection)
f

,campaign_id,date_sent,num_emails_sent,num_deliveried,num_bounced,num_opened,clicked,num_unsubscribed
0,16,2023-01-31,1,1,0,1,0,0
1,16,2023-02-20,1,1,0,1,1,0
2,16,2023-02-22,1,1,0,0,0,0
3,16,2023-02-23,1,1,0,1,0,0
4,16,2023-02-25,1,1,0,0,0,0
5,16,2023-02-26,1,1,0,1,0,0
6,16,2023-02-28,1,1,0,0,0,0
7,16,2023-03-01,1,1,0,1,0,0
8,16,2023-03-03,6,6,0,3,0,0
9,16,2023-03-05,1,1,0,1,0,0


In [162]:
query = '''
SELECT 
  *
FROM 
  deliveries d
JOIN 
  metrics m ON d.delivery_id = m.delivery_id
where campaign_id = 16 and SUBSTRING(d.created_at, 1, 10) = '2023-03-06'
order by delivery_id
'''

f = pd.read_sql_query(query, connection)
f

,index,worskpace_id,delivery_id,internal_customer_id,subject_id,event_id,delivery_type,campaign_id,action_id,newsletter_id,...,index,worskpace_id,event_id,delivery_id,metric,created_at,reason,link_id,link_url,seq_num
0,241,123456,dgT33AcAAI3eJYzeJQGGts_soqSC0stx1MLz-fI=,f7dc0714cc8e018dde25,01GTKNSJWXCF8SVV4FW81YQ93F,None,email,16,64,None,...,749,123456,01GTVD0EKP4X1EWYQT66A721YH,dgT33AcAAI3eJYzeJQGGts_soqSC0stx1MLz-fI=,opened,2023-03-06 12:05:43.000000 UTC,None,None,None,25261261
1,241,123456,dgT33AcAAI3eJYzeJQGGts_soqSC0stx1MLz-fI=,f7dc0714cc8e018dde25,01GTKNSJWXCF8SVV4FW81YQ93F,None,email,16,64,None,...,750,123456,01GTVCZW2M25Z38W23KV92XQ9S,dgT33AcAAI3eJYzeJQGGts_soqSC0stx1MLz-fI=,sent,2023-03-06 12:05:25.000000 UTC,None,None,None,25261169
2,241,123456,dgT33AcAAI3eJYzeJQGGts_soqSC0stx1MLz-fI=,f7dc0714cc8e018dde25,01GTKNSJWXCF8SVV4FW81YQ93F,None,email,16,64,None,...,751,123456,01GTVD00YSTM3WG35WKZ24EGEF,dgT33AcAAI3eJYzeJQGGts_soqSC0stx1MLz-fI=,delivered,2023-03-06 12:05:30.000000 UTC,None,None,None,25261220
3,247,123456,dgT33AcAAI3gJYzgJQGGtwS0Xrmz_VL3i3yPbdg=,f7dc0714cc90018de025,01GTKS35TX4NJGBZ41GEX2JMAY,None,email,16,64,None,...,770,123456,01GTVG9DZDXAXYJ5DSA63N0KS8,dgT33AcAAI3gJYzgJQGGtwS0Xrmz_VL3i3yPbdg=,sent,2023-03-06 13:03:04.000000 UTC,None,None,None,25261775
4,247,123456,dgT33AcAAI3gJYzgJQGGtwS0Xrmz_VL3i3yPbdg=,f7dc0714cc90018de025,01GTKS35TX4NJGBZ41GEX2JMAY,None,email,16,64,None,...,771,123456,01GTVG9ZJND74A4RKC2H6V40DB,dgT33AcAAI3gJYzgJQGGtwS0Xrmz_VL3i3yPbdg=,opened,2023-03-06 13:03:22.000000 UTC,None,None,None,25262132
5,247,123456,dgT33AcAAI3gJYzgJQGGtwS0Xrmz_VL3i3yPbdg=,f7dc0714cc90018de025,01GTKS35TX4NJGBZ41GEX2JMAY,None,email,16,64,None,...,772,123456,01GTVGKF8AP66KFZX0RCA2Y693,dgT33AcAAI3gJYzgJQGGtwS0Xrmz_VL3i3yPbdg=,opened,2023-03-06 13:08:31.000000 UTC,None,None,None,25262796
6,247,123456,dgT33AcAAI3gJYzgJQGGtwS0Xrmz_VL3i3yPbdg=,f7dc0714cc90018de025,01GTKS35TX4NJGBZ41GEX2JMAY,None,email,16,64,None,...,773,123456,01GTVG9WPNZ8ZC9BS09DXJ3S9V,dgT33AcAAI3gJYzgJQGGtwS0Xrmz_VL3i3yPbdg=,delivered,2023-03-06 13:03:19.000000 UTC,None,None,None,25262075
7,253,123456,dgT33AcAAI3hJYzhJQGGtwTyx0dF9mNGpDNWE8o=,f7dc0714cc91018de125,01GTKS3Q9PZA8Z3F1E20TRKQ2Z,None,email,16,64,None,...,785,123456,01GTVG9XM4E62EX39QMTA6EKTN,dgT33AcAAI3hJYzhJQGGtwTyx0dF9mNGpDNWE8o=,sent,2023-03-06 13:03:20.000000 UTC,None,None,None,25262087
8,253,123456,dgT33AcAAI3hJYzhJQGGtwTyx0dF9mNGpDNWE8o=,f7dc0714cc91018de125,01GTKS3Q9PZA8Z3F1E20TRKQ2Z,None,email,16,64,None,...,786,123456,01GTVGA7CBQWHEKFMEFDRTRZW6,dgT33AcAAI3hJYzhJQGGtwTyx0dF9mNGpDNWE8o=,delivered,2023-03-06 13:03:30.000000 UTC,None,None,None,25262317
9,258,123456,dgT33AcAAI3iJYziJQGGtwVA9v8i0ZgEDkg_tgs=,f7dc0714cc92018de225,01GTKS471RBRJ859BH420RXYF1,None,email,16,64,None,...,797,123456,01GTVGAH6YW3WQJJWE57PTQA89,dgT33AcAAI3iJYziJQGGtwVA9v8i0ZgEDkg_tgs=,sent,2023-03-06 13:03:41.000000 UTC,None,None,None,25262525


In [206]:
query = '''
SELECT 
  d.campaign_id,
  SUBSTRING(d.created_at, 1, 10) AS date_sent,
  COUNT(DISTINCT CASE WHEN m.metric = 'sent' THEN d.delivery_id END) AS num_emails_sent,
  COUNT(DISTINCT CASE WHEN m.metric = 'clicked' THEN d.delivery_id END) AS num_emails_clicked,
  COUNT(DISTINCT CASE WHEN m.metric = 'converted' THEN d.delivery_id END) AS num_converted
FROM 
  deliveries d
JOIN 
  metrics m ON d.delivery_id = m.delivery_id
GROUP BY 
  d.campaign_id,
  date_sent;

'''

f = pd.read_sql_query(query, connection)
f

,campaign_id,date_sent,num_emails_sent,num_emails_clicked,num_converted
0,16,2023-01-31,1,0,0
1,16,2023-02-20,1,1,0
2,16,2023-02-22,1,0,0
3,16,2023-02-23,1,0,0
4,16,2023-02-25,1,0,0
5,16,2023-02-26,1,0,0
6,16,2023-02-28,1,0,0
7,16,2023-03-01,1,0,0
8,16,2023-03-03,6,0,0
9,16,2023-03-05,1,0,0


In [218]:
query = '''
SELECT 
  d.campaign_id,
  d.action_id AS action_id,
  d.delivery_id,
  SUBSTRING(d.created_at, 1, 10) AS date_sent, 
  COUNT(DISTINCT CASE WHEN m.metric = 'sent' THEN d.delivery_id END) AS num_emails_sent, 
  COUNT(DISTINCT CASE WHEN m.metric = 'delivered' THEN d.delivery_id END) AS num_delivered,
  COUNT(DISTINCT CASE WHEN m.metric = 'opened' THEN d.delivery_id END) AS num_opened,
  COUNT(DISTINCT CASE WHEN m.metric = 'clicked' THEN d.delivery_id END) AS num_clicked,
  COUNT(DISTINCT CASE WHEN m.metric = 'unsubscribed' THEN d.delivery_id END) AS num_unsubscribed, 
  COUNT(DISTINCT CASE WHEN m.metric = 'converted' THEN d.delivery_id END) AS num_converted
FROM 
  deliveries d
JOIN 
  metrics m ON d.delivery_id = m.delivery_id
WHERE 
  d.campaign_id IN (SELECT campaign_id FROM deliveries GROUP BY campaign_id HAVING COUNT(delivery_id) > 1)
GROUP BY 
  d.campaign_id, action_id, date_sent;

'''

f = pd.read_sql_query(query, connection)
f

,campaign_id,action_id,delivery_id,date_sent,num_emails_sent,num_delivered,num_opened,num_clicked,num_unsubscribed,num_converted
0,16,61,dgT33AcAAI3eJYzeJQGGp1zLnTRgc0bH4URJgj4=,2023-03-03,3,3,2,0,0,0
1,16,61,dgT33AcAAI3lJYzlJQGGt_p_2wXco-VSy3gpLgA=,2023-03-06,2,1,1,0,0,0
2,16,61,dgT33AcAAI3xJYzxJQGGwNDCU2KxmP2zxLcUMXk=,2023-03-08,1,1,0,0,0,0
3,16,61,dgT33AcAAI6BJo2BJgGG6fr6IwjPYZYfZgZUhyM=,2023-03-16,1,1,1,0,0,0
4,16,62,dgT33AcAAI3XJYzXJQGGeZ_dPQFq6sQ-FZ8aKTA=,2023-02-22,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
78,106,986,dgT33AcAAI-2JY62JQGGzHTgAZyGyMm1lZtmQDc=,2023-03-10,15,15,8,1,0,0
79,106,986,dgT33AcAAI31JYz1JQGG3pm9nj_RG23uvTD5sEI=,2023-03-14,1,1,1,0,0,0
80,106,986,dgT33AcAAI3JCIzJCAGG646KxJwU7tS9GMJjV98=,2023-03-16,1,1,0,0,0,0
81,106,1073,dgT33AcAAI-2JY62JQGG2-f84M6sRo5E_D6ChHg=,2023-03-13,15,15,8,0,0,0
